In [33]:
import numpy as np
import pandas as pd

from collections import Counter

import logging
logging.basicConfig(filename='envSim.log', level=logging.DEBUG, filemode='w', format='[%(levelname)s] %(message)s', force=True)

rng = np.random.default_rng()

# MAIN

In [34]:
class WareHouse:

    EXPIRATION = 7
    EXPIRATION_TRASH = 3
    ITEM_ORDER_DELAY = 4

    def __init__(self, days, supplied_stores=None, start_quantity=0):
        if supplied_stores is None:
            supplied_stores = []

        self.supplied_stores = supplied_stores

        self.storage = WareHouse.create_items(start_quantity)

        self.incoming : list[pd.DataFrame] = [
            WareHouse.create_items(0) for _ in range(days)
        ]

    def update_storage(self, day_index):
        """
        Áruk megérkeznek a boltba
        """
        self.storage = pd.concat((self.storage, self.incoming[day_index]))

    def update_predictions(self, day_index, prediction_amount):
        """
        Előrejelzések módosítása
        """
        self.incoming[day_index+WareHouse.ITEM_ORDER_DELAY] = WareHouse.create_items(prediction_amount)

    def drop_expired(self):
        """
        Kidobjuk a WH_EXP_TRASH-nál kisebb értékű termékeket
        :return: kidobott termékek száma
        """

        nr_of_dropped_ = self.storage[self.storage['expiration']<WareHouse.EXPIRATION_TRASH].shape[0]

        self.storage = self.storage[self.storage['expiration']>=WareHouse.EXPIRATION_TRASH]

        return nr_of_dropped_

    def supply(self, amount):
        """
        Termékek leszállítása
        :param amount: mennyiség
        :return: leszállított termékek
        """
        supplied = self.storage.sort_values('expiration', ascending=True).head(amount)
        self.storage.drop(supplied.index, inplace=True)
        return supplied

    # noinspection PyPep8Naming
    @property
    def storageSize(self):
        """
        :return: Raktár készlet nagysága
        """
        return self.storage.shape[0]

    @staticmethod
    def create_items(amount):
        return pd.DataFrame(
            {'expiration': [WareHouse.EXPIRATION]*amount}
        )


class Store:

    EXPIRATION_START = 1
    EXPIRATION_END = 5

    def __init__(self, days, start_quantity=0):
        self.predictions = [{'center' : rng.uniform(80,120), 'scale' : rng.uniform(10,30)} for _ in range(days)]
        self.storage = pd.DataFrame(
            {'expiration':
                [rng.uniform(Store.EXPIRATION_START, Store.EXPIRATION_END) for _ in range(start_quantity)]
            })

    def get_order(self, day_index):
        return round(self.predictions[day_index]['center'])

    def get_demand(self, day_index):
        prediction = self.predictions[day_index]
        demand = rng.normal(prediction['center'], prediction['scale'])
        while demand < 0:
            demand = rng.normal(demand)

        return round(demand)

    def sell(self, amount):
        if amount > self.storageSize: raise ValueError("Nincs ennyi termék a boltban")


        for i in range(amount):
            current_sale = self.get_random_expiration()
            while current_sale not in list(self.storage.expiration):
                current_sale = self.get_random_expiration()

            pos = self.storage[self.storage['expiration'] == current_sale].index[0]

            self.storage.drop(pos, inplace=True)


    def drop_expired(self):
        """
        Kidobjuk a lejárt termékeket
        :return: kidobott termékek száma
        """

        nr_of_dropped_ = self.storage[self.storage['expiration']<1].shape[0]
        self.storage = self.storage[self.storage['expiration']>=1]

        return nr_of_dropped_

    # noinspection PyPep8Naming
    @property
    def storageSize(self):
        """
        :return: Bolt készletének nagysága
        """
        return self.storage.shape[0]

    def get_random_expiration(self):
        """adott min es max lejarattal generalunk vasarloi eloszlast
        ez itt: sokan vesznek hamar lejarot (akciok miatt)
        sokan vesznek friss termeket
        ket random eloszlas osszeget vesszuk"""

        f1 = min(self.storage['expiration'])
        f2 = max(self.storage['expiration'])
        customer_demand_dist = list(filter(lambda x: (f1 <= x <= f2), list(np.random.normal(f1, (f2 - f1) * 0.1, 50)) + list(np.random.normal(f2, (f2 - f1) // 3, 100))))
        return round(np.random.choice(customer_demand_dist))

In [35]:
def run_simulation(days, nr_stores):
    costs = Counter()

    stores = [Store(days) for _ in range(nr_stores)]
    warehouse = WareHouse(days, stores, 1000)

    for current_day in range(days):

        logging.debug('-'*50+f" {current_day+1}. NAP "+"-"*50)

        warehouse.update_storage(current_day)


        if current_day + WareHouse.ITEM_ORDER_DELAY < days:
            # RAKTÁRi RENDELÉS
            storage_order = 0
            for store in stores:
                storage_order += round(store.predictions[current_day+WareHouse.ITEM_ORDER_DELAY]['center'])

            warehouse.update_predictions(current_day, storage_order)
            logging.debug(f"Rendel a raktár: {storage_order}-t a {current_day+WareHouse.ITEM_ORDER_DELAY+1}. napra.")



        # BOLTI KISZÁLLÍTÁS, ELADÁS
        for (i, store) in enumerate(stores, start=1):
            logging.debug(f"{'-'*10} {i}. Bolt {'-'*10}")

            # BOLTI KISZÁLLÍTÁS
            current_order = store.get_order(current_day)
            final_order = current_order
            logging.debug(f"Ennyit rendel a bolt mara: {current_order}")

            if current_order > warehouse.storageSize:
                logging.debug("A raktár erre a boltra nem tudja teljesíteni a kiszállítást!")
                final_order = warehouse.storageSize
                logging.debug(f'A raktár csak ennyit szállít ki: {final_order}')
                logging.debug(f"COST: {current_order-final_order}")

                costs['not_enough_in_warehouse'] += (current_order-final_order)



            store.storage = pd.concat((store.storage, warehouse.supply(final_order)))
            store.storage.reset_index(inplace=True, drop=True)


            # BOLTI ELADÁS
            demand = store.get_demand(current_day)

            logging.debug(f"Bolti keszlet: {store.storageSize}")
            logging.debug(f"Ennyit akarnak venni: {demand}")


            if demand > store.storageSize:
                logging.debug("A boltban nincs elég termék")
                logging.debug(f"COST: {demand-store.storageSize}")
                costs['not_enough_in_store'] += (demand-store.storageSize)

            current_sale = min(store.storageSize, demand)

            store.sell(current_sale)
            logging.debug(f"Ennyi terméket adtak el: {current_sale}")


            # BOLTI LEJÁRT TERMÉKEK KIDOBÁSA
            store.storage['expiration'] -= 1
            nr_of_dropped = store.drop_expired()

            logging.debug(f"HIBA (bolti kidobás): {nr_of_dropped}")
            costs['store_throwout'] += nr_of_dropped

            logging.debug(f"Bolti keszlet: {store.storageSize}")


        # RAKTÁRI LEJÁRT TERMÉKEK KIDOBÁSA
        logging.debug('-'*25)
        warehouse.storage['expiration'] -= 1
        nr_of_dropped = warehouse.drop_expired()

        logging.debug(f"HIBA (raktari kidobás): {nr_of_dropped}")
        costs['warehouse_throwout'] += nr_of_dropped
        logging.debug(f"Raktár készlet: {warehouse.storageSize}")

        logging.debug(costs)


    return costs

In [36]:
simulation_costs = run_simulation(
    days = 10,
    nr_stores = 3
)

cost_weights = {
    'not_enough_in_store' : 1,
    'not_enough_in_warehouse' : 1,
    'store_throwout' : 5,
    'warehouse_throwout': 3
}

for cost_type in cost_weights:
    logging.info(f"{cost_type} : {simulation_costs[cost_type]}")
    print(f"{cost_type} : {simulation_costs[cost_type]}")


not_enough_in_store : 421
not_enough_in_warehouse : 234
store_throwout : 0
warehouse_throwout : 0
